In [3]:
from ultralytics import YOLO, checks, hub
checks()

hub.login('65bbc9483082eef4207b18d12e40c86ce9774548c9')

model = YOLO('https://hub.ultralytics.com/models/m9MdEb71tJL5SRCTmDFA')
#results = model.train()

Ultralytics YOLOv8.2.14  Python-3.10.8 torch-1.13.1+cpu CPU (12th Gen Intel Core(TM) i7-12700H)
Setup complete  (20 CPUs, 31.7 GB RAM, 725.2/930.9 GB disk)
requirements: Ultralytics requirement ['hub-sdk>=0.0.6'] not found, attempting AutoUpdate...

requirements: AutoUpdate success  1.8s, installed 1 package: ['hub-sdk>=0.0.6']
requirements:  Restart runtime or rerun command for updates to take effect



ModuleNotFoundError: No module named 'hub_sdk'

In [ ]:
from ultralytics import YOLO
import pandas as pd
import os
from tqdm import tqdm
import numpy as np
from glob import glob
import cv2
CLASSES = [
    'finger-1', 'finger-2', 'finger-3', 'finger-4', 'finger-5',
    'finger-6', 'finger-7', 'finger-8', 'finger-9', 'finger-10',
    'finger-11', 'finger-12', 'finger-13', 'finger-14', 'finger-15',
    'finger-16', 'finger-17', 'finger-18', 'finger-19', 'Trapezium',
    'Trapezoid', 'Capitate', 'Hamate', 'Scaphoid', 'Lunate',
    'Triquetrum', 'Pisiform', 'Radius', 'Ulna',
]
CLASS2IND = {v: i for i, v in enumerate(CLASSES)}
IND2CLASS = {v: k for k, v in CLASS2IND.items()}
def rle_encode(mask):
    pixels = mask.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)


def inference():
    model = YOLO(r'E:\프로젝트4\yolo_is_coming\runs\segment\train2\weights\best.pt').cuda()
    infer_images = sorted(glob(r"E:/프로젝트4/data/data/test/*/*/*.png"))
    
    rles = []
    filename_and_class = []

    for idx, infer_image in tqdm(enumerate(infer_images)):
        result = model.predict(infer_image, imgsz=2048)[0]
        boxes = result.boxes.data.cpu().numpy()
        scores, classes = boxes[:, 4].tolist(), boxes[:, 5].astype(np.uint8).tolist()
        masks = result.masks.xy
        
        datas = [[a, b, c] for a, b, c, in zip(classes, scores, masks)]
        datas = sorted(datas, key=lambda x: (x[0], -x[1]))
        img_name = infer_image.split("/")[-1]
        
        is_checked = [False] * 29
        csv_idx, data_idx = 0, 0
        while data_idx < len(datas):
            c, s, mask_pts = datas[data_idx]
            if is_checked[c]:
                data_idx += 1
                continue
            
            empty_mask = np.zeros((2048, 2048), dtype=np.uint8)
            if c == csv_idx:
                is_checked[c] = True
                pts = [[int(x[0]), int(x[1])] for x in mask_pts]
                cv2.fillPoly(empty_mask, [np.array(pts)], 1)
                rle = rle_encode(empty_mask)
                rles.append(rle)
                filename_and_class.append(f"{IND2CLASS[c]}_{img_name}")
                data_idx += 1
            else:
                rle = rle_encode(empty_mask)
                rles.append(rle)
                filename_and_class.append(f"{IND2CLASS[csv_idx]}_{img_name}")
            csv_idx += 1
            
    classes, filename = zip(*[x.split("_") for x in filename_and_class])
    image_name = [os.path.basename(f) for f in filename]
    df = pd.DataFrame({
        "image_name": image_name,
        "class": classes,
        "rle": rles,
    })

    if not os.path.exists('./result'):                                                           
        os.makedirs('./result')
    f_name = 'yolov11output.csv'
    df.to_csv(os.path.join('result', f_name), index=False)
        
    
if __name__ == '__main__':
    inference()

0it [00:00, ?it/s]


image 1/1 E:\4\data\data\test\DCM\ID040\image1661319116107.png: 2048x2048 1 finger-1, 1 finger-2, 1 finger-3, 1 finger-4, 1 finger-5, 1 finger-6, 1 finger-7, 1 finger-8, 1 finger-9, 1 finger-10, 1 finger-11, 1 finger-12, 1 finger-13, 1 finger-14, 1 finger-15, 1 finger-16, 1 finger-17, 1 finger-18, 1 finger-19, 1 Trapezium, 1 Trapezoid, 1 Capitate, 1 Hamate, 1 Scaphoid, 1 Lunate, 1 Triquetrum, 1 Pisiform, 1 Radius, 1 Ulna, 6121.1ms
Speed: 15.0ms preprocess, 6121.1ms inference, 238.0ms postprocess per image at shape (1, 3, 2048, 2048)


1it [00:13, 13.75s/it]


image 1/1 E:\4\data\data\test\DCM\ID040\image1661319145363.png: 2048x2048 1 finger-1, 1 finger-2, 1 finger-3, 1 finger-4, 1 finger-5, 1 finger-6, 1 finger-7, 1 finger-8, 1 finger-9, 1 finger-10, 1 finger-11, 1 finger-12, 1 finger-13, 1 finger-14, 1 finger-15, 1 finger-16, 1 finger-17, 1 finger-18, 1 finger-19, 1 Trapezium, 1 Trapezoid, 1 Capitate, 1 Hamate, 1 Scaphoid, 1 Lunate, 1 Triquetrum, 1 Pisiform, 1 Radius, 1 Ulna, 5769.6ms
Speed: 22.0ms preprocess, 5769.6ms inference, 205.0ms postprocess per image at shape (1, 3, 2048, 2048)


2it [00:20,  9.63s/it]

...

286it [32:20,  6.76s/it]


image 1/1 E:\4\data\data\test\DCM\ID550\image1667354405140.png: 2048x2048 1 finger-1, 1 finger-2, 1 finger-3, 1 finger-4, 1 finger-5, 1 finger-6, 1 finger-7, 1 finger-8, 1 finger-9, 1 finger-10, 1 finger-11, 1 finger-12, 1 finger-13, 1 finger-14, 1 finger-15, 1 finger-16, 1 finger-17, 1 finger-18, 1 finger-19, 1 Trapezium, 1 Trapezoid, 1 Capitate, 1 Hamate, 1 Scaphoid, 1 Lunate, 1 Triquetrum, 1 Pisiform, 1 Radius, 1 Ulna, 5883.1ms
Speed: 16.0ms preprocess, 5883.1ms inference, 213.1ms postprocess per image at shape (1, 3, 2048, 2048)


287it [32:27,  6.77s/it]


image 1/1 E:\4\data\data\test\DCM\ID550\image1667354424553.png: 2048x2048 1 finger-1, 1 finger-2, 1 finger-3, 1 finger-4, 1 finger-5, 1 finger-6, 1 finger-7, 1 finger-8, 1 finger-9, 1 finger-10, 1 finger-11, 1 finger-12, 1 finger-13, 1 finger-14, 1 finger-15, 1 finger-16, 1 finger-17, 1 finger-18, 1 finger-19, 1 Trapezium, 1 Trapezoid, 1 Capitate, 1 Hamate, 1 Scaphoid, 1 Lunate, 1 Triquetrum, 1 Pisiform, 1 Radius, 1 Ulna, 5797.6ms
Speed: 14.0ms preprocess, 5797.6ms inference, 217.1ms postprocess per image at shape (1, 3, 2048, 2048)


288it [32:34,  6.79s/it]
